In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_excel("bacteria.xlsx")
#data = data.iloc[:,1:2]
data.head()

,UHID,Patients Name,RAMJA Results
0,20220065432,Sahroj,Enterococcus faecalis
1,20240016182,Shyanu,NaN
2,20230125389,Saloni,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman..."
3,20180022685,Ravita,"K.pneumoniae, E. faecalis, P. mirabilis, A. b..."
4,20201415102,Jyoti,NaN


In [11]:
data.shape

(146, 3)

In [10]:
bacteria[0].upper()

'ENTEROCOCCUS FAECALIS'

In [13]:
data = data.dropna()

In [16]:
data.head()

,UHID,Patients Name,RAMJA Results
0,20220065432,Sahroj,Enterococcus faecalis
2,20230125389,Saloni,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman..."
3,20180022685,Ravita,"K.pneumoniae, E. faecalis, P. mirabilis, A. b..."
5,20240012395,kedari Devi,"E.coli, A. baumannii, P.mirabilis, E. faecalis..."
6,20220166372,Babita Devi,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus"


In [47]:
def cleanup(name):
    name = name.replace("."," ")
    name = name.replace(r'\s+', ' ',regex = True)
    name = name.strip()
    name = name.upper()
    return name

In [48]:
data['RAMJA Results'].apply(cleanup)
#data['RAMJA Results'].str.replace(".","")


TypeError: str.replace() takes no keyword arguments

In [31]:
bacteria
df['Column'] = df['Column'].str.replace(r'\s+', '', regex=True)

0                                  ENTEROCOCCUS FAECALIS
2      E.COLI, K.PNEUMONIAE, P. AERUGINOSA, A. BAUMAN...
3      K.PNEUMONIAE, E. FAECALIS, P. MIRABILIS,  A. B...
5      E.COLI, A. BAUMANNII, P.MIRABILIS, E. FAECALIS...
6        PSEUDOMONAS, P.MIRABILIS, E.FAECALIS, S. AUREUS
                             ...                        
141                                              STERILE
142                                              STERILE
143                                              STERILE
144                                      S SAPROPHYTICUS
145                                              STERILE
Name: RAMJA Results, Length: 143, dtype: object

In [35]:
bacteria.str.split(',', expand=True)

,0,1,2,3,4,5,6
0,ENTEROCOCCUS FAECALIS,None,None,None,None,None,None
2,E.COLI,K.PNEUMONIAE,P. AERUGINOSA,A. BAUMANNI AND S. AUREUS,None,None,None
3,K.PNEUMONIAE,E. FAECALIS,P. MIRABILIS,A. BAUMANNI AND S. AUREUS,None,None,None
5,E.COLI,A. BAUMANNII,P.MIRABILIS,E. FAECALIS,S AUREUS,None,None
6,PSEUDOMONAS,P.MIRABILIS,E.FAECALIS,S. AUREUS,None,None,None
...,...,...,...,...,...,...,...
141,STERILE,None,None,None,None,None,None
142,STERILE,None,None,None,None,None,None
143,STERILE,None,None,None,None,None,None
144,S SAPROPHYTICUS,None,None,None,None,None,None
